In [1]:
import os
from dotenv import load_dotenv
import wget

In [2]:
env = load_dotenv()
LLAMA_API_KEY = os.getenv('LLAMA_CLOUD_API_KEY')
OPENAI_KEY = os.getenv('OPEN_AI_API_KEY')
HF_KEY = os.getenv('HF_KEY')

In [3]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openai import OpenAI

from llama_index.core import Settings



embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
llm = OpenAI(model="gpt-3.5-turbo", api_key= OPENAI_KEY)

Settings.embed_model = embed_model
Settings.llm = llm


In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
# urls = [
#     "https://openreview.net/pdf?id=VtmBAGCN7o",
#     "https://openreview.net/pdf?id=6PmJoRfdaK",
#     "https://openreview.net/pdf?id=hSyW5go0v8",
# ]

# struc_doc_urls =[
#     "https://www.opm.gov/forms/pdfimage/sf50.pdf"
# ]

# files = []

# all_urls = urls + struc_doc_urls
# for url in all_urls:
#     file = wget.download(url=url)
#     files.append(file)


In [6]:
from PIL import Image
from llama_index.core import SimpleDirectoryReader

def parse_image(file_path):
    image = Image.open(file_path)
    text = LlamaParse.parse_image(image)
    return text

def base_reader(file_path):
    reader = LlamaParse(result_type = 'text')
    base_docs = reader.load_data(file_path)
    return base_docs

In [7]:
files = [
    "travel_doc_sample2.pdf",
    "travel_doc_sample.pdf",
]

In [9]:
from llama_parse import LlamaParse
from llama_index.core.schema import Document

def _load(file_path: str) -> Document:
    parser = LlamaParse(result_type='text')
    json_response = parser.get_json_result(file_path)
    page_results = json_response[0]['pages']
    docs = []
    for item in page_results:
        doc = Document(
                text=item.get("text"), 
                metadata={"page_content":item["page"]}
            )
    docs.append(doc)
    return docs

In [19]:
from llama_index.core import VectorStoreIndex, SummaryIndex
from llama_index.core.node_parser import SentenceSplitter, LlamaParseJsonNodeParser
from llama_index.core.tools import FunctionTool, QueryEngineTool, ToolMetadata
from llama_index.core.vector_stores import MetadataFilters, FilterCondition
from typing import List, Optional
from pathlib import Path

def get_vec_tools(file_path:str) -> str:
    docs = _load(file_path)
    splitter = SentenceSplitter(chunk_size=1024)
    nodes = splitter.get_nodes_from_documents(docs)
    doc_index = VectorStoreIndex(nodes = nodes)

    query_engine = doc_index.as_query_engine(similarity_top_k=5, verbose=True)

    name = Path(file_path).stem.split(" ")[0]
    query_tool = QueryEngineTool(
        query_engine = query_engine,
        metadata = ToolMetadata(
            name = f'vector_tool_{name}',
            description = (
                f'Provides information about {name}.'
                'Use a detailed plain text question as input to the tool.'
            )
        )
    )
    return query_tool
        

In [20]:
paper_to_tools_dict = {}
for file in files:
    file_query_tool = get_vec_tools(file)
    paper_to_tools_dict[file] = file_query_tool

Started parsing the file under job_id cac11eca-59de-452e-af3e-70f2663c6dbd
Started parsing the file under job_id cac11eca-ba5c-4390-a9b8-d3a580cb3766


In [21]:
from llama_index.core.agent import ReActAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = ReActAgentWorker.from_tools(
    [tool for tool in paper_to_tools_dict.values()],
    # llm=llm,
    verbose=True
)
agent = AgentRunner(agent_worker)

In [24]:
question_types = ['name of traveler', 'departing street address', 'dates of travel']

In [28]:
for file in files:
    prompt = f'In {file}, provide '
    for i, question in enumerate(question_types):
        if i == len(question_types)-1:
            prompt += 'and ' + question
        else:
            prompt += question + ', '   
    response = agent.query(prompt)
    print(str(response))

Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: vector_tool_travel_doc_sample2
Action Input: {'input': 'Name of traveler, departing street address, and dates of travel in travel_doc_sample2.pdf'}
Observation: Johnathan Doe, 222 North St, 2/1/2024 - 2/10/2024
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The name of the traveler is Johnathan Doe, the departing street address is 222 North St, and the dates of travel are from 2/1/2024 to 2/10/2024.
The name of the traveler is Johnathan Doe, the departing street address is 222 North St, and the dates of travel are from 2/1/2024 to 2/10/2024.
Thought: The user is asking for specific information from a travel document. I can use the tools to extract the required details.
Action: vector_tool_travel_doc_sample
Action Input: {'input': 'Please provide the name of the traveler, departing street address, and dates of travel from 